# Instalamos las dependencias
Necesitamos numpy y pandas

In [9]:
#!conda install -y numpy pandas 
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import numpy as np
import pandas as pd
import json

class ProfileGenerator:
    
    def __init__(self):
        self.profiles = {}
        
    def get_np(self, cpu, com):
        if cpu < 1e9 and com < 1e9:
            return 1
        elif cpu > 1e12 and com > 1e12:
            return 4
        else:
            return 2
        
    def generate_low_profile(self):
        cpu = np.random.uniform(1e7, 1e9)
        com = np.random.uniform(1024, 1e6)
        np_val = self.get_np(cpu, com)
        walltime = np.random.uniform(5*60, 15*60)  # 5-15 minutos
        profile_name = f"low_{len(self.profiles) + 1}"
        self.profiles[profile_name] = {
            "type": "parallel_homogeneous",
            "cpu": cpu,
            "com": com,
            "np": np_val,
            "walltime": walltime
        }
        return profile_name

    def generate_med_profile(self):
        if np.random.choice([True, False]):
            cpu = np.random.uniform(1e7, 1e9)
            com = np.random.uniform(1e9, 1e12)
        else:
            cpu = np.random.uniform(1e12, 2e12)
            com = np.random.uniform(1024, 1e6)
        np_val = self.get_np(cpu, com)
        walltime = np.random.uniform(30*60, 2*3600)  # 30 minutos - 2 horas
        profile_name = f"med_{len(self.profiles) + 1}"
        self.profiles[profile_name] = {
            "type": "parallel_homogeneous",
            "cpu": cpu,
            "com": com,
            "np": np_val,
            "walltime": walltime
        }
        return profile_name

    def generate_high_profile(self):
        cpu = np.random.uniform(1e12, 2e12)
        com = np.random.uniform(1e12, 2e12)  # Corregido para asegurar valores HIGH
        np_val = self.get_np(cpu, com)
        walltime = np.random.uniform(3*3600, 12*3600)  # 3-12 horas
        profile_name = f"high_{len(self.profiles) + 1}"
        self.profiles[profile_name] = {
            "type": "parallel_homogeneous",
            "cpu": cpu,
            "com": com,
            "np": np_val,
            "walltime": walltime
        }
        return profile_name

    def generate(self, number_of_profiles=10, low_percent=0.3, med_percent=0.4, high_percent=0.3):
        num_low = int(number_of_profiles * low_percent)
        num_med = int(number_of_profiles * med_percent)
        num_high = number_of_profiles - num_low - num_med

        for _ in range(num_low):
            self.generate_low_profile()

        for _ in range(num_med):
            self.generate_med_profile()

        for _ in range(num_high):
            self.generate_high_profile()
        
        return self.profiles

In [11]:
generator = ProfileGenerator()
profiles = generator.generate(50)
with open('profiles.json', 'w') as file:
    json.dump(profiles, file)

df = pd.DataFrame(profiles).T.reset_index()
df['task_type'] = df['index'].str.split('_').str[0]
profile_counts = df.groupby('task_type').size()
percentages = profile_counts / len(df) * 100
summary = pd.DataFrame({
    'Task Type': percentages.index,
    'Percentage': percentages.values
})

print("\nGenerated Profiles Percentage:")
print("-------------------------------")
print(summary)


Generated Profiles Percentage:
-------------------------------
  Task Type  Percentage
0      high        30.0
1       low        30.0
2       med        40.0
